In [1]:
import re
import pandas as pd
from tqdm import tqdm

In [2]:
with open('big.txt','r') as fd:
    lines = fd.readlines()
    words = []
    for line in lines:
        words += re.findall('\w+',line.lower())
        


In [3]:
print(len(words))
vocab = list(set(words))
print(len(vocab))

1115585
32198


In [4]:
print(vocab)

['renomination', 'galleries', 'informing', 'united', 'unabashed', '_extravasation_', 'andover', 'alluvial', 'cavalry', 'bandoliers', 'gap', 'supervenes', 'claimant', 'despised', 'introduces', 'blister', 'patroness', 'sadly', 'overcrowding', 'seek', 'cook', 'blundering', 'knaves', '_hydrophobia_', 'haitian', 'marched', 'unclasping', 'clinical', 'module', 'conclusively', 'outlining', 'explained', 'amount', 'supinator', 'mckinley', 'numbed', 'atheromatous', 'pierce', 'holocausts', 'posture', 'lustrous', 'opsonin', 'unicorn', 'pinnacles', 'outweighs', 'intruding', 'severer', 'thwee', 'connective', 'synchondrosis', 'muse', 'sob', 'couture', 'headlong', 'dams', 'david', 'cricket', 'waggon', 'caraffe', 'feet', 'clambered', 'spilt', '_wet', 'barrel', 'penthouses', 'learning', 'alluringly', 'violins', 'pieces', 'sombre', 'duke', 'abandoned', 'cheapest', 'kolyazin', 'afghanistan', 'talkative', 'there', 'tillman', 'neutralising', 'imbecile', 'venae', 'wethersfield', 'sherman', '272', 'riddle', '7

In [5]:
word_probability = {}

for word in tqdm(vocab):
    word_probability[word] = float(words.count(word)/len(words))

100%|██████████| 32198/32198 [08:10<00:00, 65.68it/s]


In [6]:
def split(word):  
    parts = []
    for i in range(len(word) + 1):
        parts += [(word[ : i], word[i : ])]
    return parts

In [7]:
def delete(word):
    
    output = []
    for l,r in split(word):
        output.append(l + r[1:])
    return output

delete('loave')

['oave', 'lave', 'love', 'loae', 'loav', 'loave']

In [8]:
def swap(word):
        
    output = []    
    for l,r in split(word):
        if (len(r) > 1):
            output.append(l + r[1] + r[0] + r[2:])
    return output
            
swap('hlelo')

['lhelo', 'hello', 'hlleo', 'hleol']

In [9]:
def replace(word):
    
    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []    

    for l,r in split(word):
        for char in characters:
            output.append(l + char +  r[1:])
    return output

len(replace('lave'))

130

In [10]:
def insert(word):

    characters = 'abcdefghijklmnopqrstuvwxyz'
    output = []

    for l,r in split(word):
        for char in characters:
            output.append(l + char + r)

    return output

len(insert('lve'))

104

In [11]:
def edit(word):   
    return list(set(insert(word) + delete(word) + swap(word) + replace(word)))

In [12]:
def spell_check_edit_1(word, count = 5):
    
    output = []
    suggested_words = edit(word)
    
    for wrd in suggested_words:        
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])

    df = pd.DataFrame(output, columns=['word', 'prob'])
    top_words = df.sort_values(by='prob', ascending=False).head(count)['word'].values
            
    return top_words

In [13]:
spell_check_edit_1('famili')

array(['family'], dtype=object)

In [14]:
def spell_check_edit_2(word, count = 5):
    
    output = []
    suggested_words = edit(word)       # Level one Edit
    
    for e1 in edit(word):
        suggested_words += edit(e1)    # Second Level Edit 
    
    suggested_words = list(set(suggested_words))
    
    for wrd in suggested_words:
        if wrd in word_probability.keys():
            output.append([wrd, word_probability[wrd]])
    return list(pd.DataFrame(output, columns = ['word','prob']).sort_values(by = 'prob', ascending = False).head(count)['word'].values)
        
spell_check_edit_2('fameli')

['family', 'namely', 'fame', 'amelie', 'camelia']